In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# html_review=[]
# for word in range(len(df['review'])):
#     cleantext = BeautifulSoup(df['review'][word], "lxml").text
#     html_review.append(cleantext)

html_review=[BeautifulSoup(df['review'][word],"lxml").text for word in range(len(df['review']))]

In [ ]:
df['html_review']=html_review

In [ ]:
df=df.drop(['html_review'],axis=1)

In [ ]:
df.head()

In [ ]:
msg=html_review[0]

In [ ]:
msg

In [ ]:
import re

In [ ]:
# processed_review=[]
# for word in range(len(html_review)):
#     msg = re.sub('[^a-zA-Z]', ' ', msg).lower()
#     processed_review.append(msg)
    
processed_review = [re.sub('[^a-zA-Z]', ' ', msg).lower() for word in range(len(html_review))]

In [ ]:
import nltk

In [ ]:
# msg = html_review[0]
# msg = re.sub('[^a-zA-Z]', ' ', msg).lower()
# list_of_word = msg.split()
# final_word = [word for word in list_of_word if not word in nltk.corpus.stopwords.words('english')]
# final_word

In [ ]:
def processed_review(msg):
    msg = re.sub('[^a-zA-Z]', ' ', msg).lower()
    list_of_word = msg.split()
    final_word = [word for word in list_of_word if not word in nltk.corpus.stopwords.words('english')]
    return final_word

In [ ]:
df['html_review'] = html_review
df.head()

In [ ]:
df['processed_review'] = df['html_review'].apply(processed_review)
df.head()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenize = Tokenizer(num_words=20000)
tokenize.fit_on_texts(df['processed_review'])
seq_data = tokenize.texts_to_sequences(df['processed_review'])

In [ ]:
seq_data_pad = pad_sequences(seq_data, maxlen=150)

In [ ]:
df['processed_review'] = df['html_review'].apply(processed_review)
df.head()

In [ ]:
df['sentiment'] = df['sentiment'].apply(lambda X:1 if X=='positive' else 0 )
df.head()

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Embedding(20000,50,input_shape=(150,)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import pickle
pickle.dump(seq_data_pad, open('seq_data_imdb.pkl','wb'))

In [ ]:
model.fit(seq_data_pad, y, batch_size=128, epochs=5)

In [ ]:
import tensorflow
# write the function to predict given review is positive or negative
new_model = tensorflow.keras.models.load_model('')

In [ ]:
def get_class(review):   
    review = preprocess(review)
    review = tokenize.texts_to_sequences([review])
    review=pad_sequences(review, maxlen=150)
    s = model.predict(review)[0][0]
    return 'positive' if s>=0.5 else 'negative'

In [ ]:
msg = 'Best movie ever shooted in India.... First of all let me tell that I love to watch horror and fictional movies... I love both the movies of Bahubali series; the beginning and conclusion(Bahubali 2)..But Bahubali 2 is best between these 2... The acting, the sound, bass all are simply amazing... Prabhas, Rana, Tamanna and Anushka all acted exceptionally well and especially I am addicted to BHALLALDEV character....👿 Its a must watch movie for all generations'
get_class(msg)